In [84]:
import scipy
from PIL import Image
from scipy import ndimage
import skimage
from dnn_utils_v2 import *
import numpy as np
import h5py
import matplotlib.pyplot as plt
from testCases_v4  import *
from dnn_utils_v2 import sigmoid , sigmoid_backward , relu , relu_backward

In [85]:
%matplotlib inline
plt.rcParams['figure.figsize'] = (5.0 , 4.0)
plt.rcParams['image.interpolation']= 'nearest'
plt.rcParams['image.cmap']= 'gray'

%reload_ext autoreload
%autoreload 2

np.random.seed(1)

In [146]:
#inicializar parametros
def initialize_parameters_deep(layer_dims):
    np.random.seed(3)
    parameters = {}
    L = len(layer_dims)
    for i in range(1,L):
        parameters["W"+str(i)] =  np.random.randn(layer_dims[int(i)] ,layer_dims[int(i)-1]) *0.01
        parameters["b"+str(i)] =  np.zeros((layer_dims[int(i)] , 1))
      
        assert(parameters['W' + str(i)].shape == (layer_dims[int(i)], layer_dims[int(i)-1]))
        assert(parameters['b' + str(i)].shape == (layer_dims[int(i)], 1))
    return parameters


In [147]:
#Probando la función
parameters = initialize_parameters_deep([5,4,3])
print("W1 = " + str(parameters["W1"]))
print("b1 = " + str(parameters["b1"]))
print("W2 = " + str(parameters["W2"]))
print("b2 = " + str(parameters["b2"]))


W1 = [[ 0.01788628  0.0043651   0.00096497 -0.01863493 -0.00277388]
 [-0.00354759 -0.00082741 -0.00627001 -0.00043818 -0.00477218]
 [-0.01313865  0.00884622  0.00881318  0.01709573  0.00050034]
 [-0.00404677 -0.0054536  -0.01546477  0.00982367 -0.01101068]]
b1 = [[0.]
 [0.]
 [0.]
 [0.]]
W2 = [[-0.01185047 -0.0020565   0.01486148  0.00236716]
 [-0.01023785 -0.00712993  0.00625245 -0.00160513]
 [-0.00768836 -0.00230031  0.00745056  0.01976111]]
b2 = [[0.]
 [0.]
 [0.]]


In [148]:
def linear_forward(A, W, b):
    Z = np.dot(W,A)+b
    assert(Z.shape == (W.shape[0], A.shape[1]))
    cache = (A, W, b)
    return Z, cache

In [149]:
#Probando la función
A, W, b = linear_forward_test_case()
Z, linear_cache = linear_forward(A, W, b)
print("Z = " + str(Z))

Z = [[ 3.26295337 -1.23429987]]


In [150]:
def linear_activation_forward(A_prev, W, b, activation):
    if activation == "sigmoid":
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = sigmoid(Z)
    
    elif activation == "relu":
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = relu(Z)
    
    assert (A.shape == (W.shape[0], A_prev.shape[1]))
    cache = (linear_cache, activation_cache)
    return A, cache

In [91]:
#Probando a la función
A_prev, W, b = linear_activation_forward_test_case()
A, linear_activation_cache = linear_activation_forward(A_prev, W, b, activation = "sigmoid")
print("With sigmoid: A = " + str(A))
A, linear_activation_cache = linear_activation_forward(A_prev, W, b, activation = "relu")
print("With ReLU: A = " + str(A))

With sigmoid: A = [[0.96890023 0.11013289]]
With ReLU: A = [[3.43896131 0.        ]]


In [135]:
def L_model_forward(X, parameters):
    caches = []
    A = X
    L = len(parameters) // 2
    caches = list()
    for l in range(1, L):
        A_prev = A
        A, cache = linear_activation_forward(A_prev, parameters["W" + str(l)], parameters["b" + str(l)], activation='relu')
        caches.append(cache)
    AL, cache = linear_activation_forward(A, parameters["W" + str(L)], parameters["b" + str(L)], activation='sigmoid')
    caches.append(cache)
    assert(AL.shape == (1,X.shape[1]))
    return AL, caches 

In [93]:
X, parameters = L_model_forward_test_case_2hidden()
AL, caches = L_model_forward(X, parameters)
print("AL = " + str(AL))
print("Length of caches list = " + str(len(caches)))

AL = [[0.03921668 0.70498921 0.19734387 0.04728177]]
Length of caches list = 3


In [94]:
def compute_cost (AL,Y):
    m = Y.shape[1]
    
    cost = ( np.sum( np.dot( Y,np.log(AL).T ) + np.dot( ( 1-Y ),( np.log(1-AL).T ) ) ) )/(-m)
    
    cost = np.squeeze(cost)
    assert( cost.shape == () )
    return cost

In [95]:
Y, AL = compute_cost_test_case()
print("cost = " + str(compute_cost(AL, Y)))

cost = 0.414931599615397


In [96]:
def linear_backward(dZ, cache):
    A_prev, W, b = cache
    m = A_prev.shape[1]
    
    
    dW = ( np.dot( dZ,A_prev.T) )/(m)
    db = ( np.sum(dZ , axis=1, keepdims=True) )/(m)
    dA_prev = np.dot( W.T,dZ )

    assert (dA_prev.shape == A_prev.shape)
    assert (dW.shape == W.shape)
    assert (db.shape == b.shape)

    return dA_prev, dW, db

In [97]:
#Probando la función
dZ, linear_cache = linear_backward_test_case()
dA_prev, dW, db = linear_backward(dZ, linear_cache)
print ("dA_prev = "+ str(dA_prev))
print ("dW = " + str(dW))
print ("db = " + str(db))


dA_prev = [[ 0.51822968 -0.19517421]
 [-0.40506361  0.15255393]
 [ 2.37496825 -0.89445391]]
dW = [[-0.10076895  1.40685096  1.64992505]]
db = [[0.50629448]]


In [98]:
def linear_activation_backward(dA, cache, activation):
    linear_cache, activation_cache = cache
    
    if activation == "relu":
        dZ =  relu_backward (dA, activation_cache)
        dA_prev, dW, db = linear_backward(dZ, linear_cache)
    
    elif activation == "sigmoid":
        dZ = sigmoid_backward (dA, activation_cache)
        dA_prev, dW, db = linear_backward(dZ, linear_cache)
    
    return dA_prev, dW, db

In [99]:
#Probando la función
dAL, linear_activation_cache = linear_activation_backward_test_case()
dA_prev, dW, db = linear_activation_backward(dAL, linear_activation_cache, activation = "sigmoid")
print ("sigmoid:")
print ("dA_prev = "+ str(dA_prev))
print ("dW = " + str(dW))
print ("db = " + str(db) + "\n")
dA_prev, dW, db = linear_activation_backward(dAL, linear_activation_cache, activation = "relu")
print ("relu:")
print ("dA_prev = "+ str(dA_prev))
print ("dW = " + str(dW))
print ("db = " + str(db))

sigmoid:
dA_prev = [[ 0.11017994  0.01105339]
 [ 0.09466817  0.00949723]
 [-0.05743092 -0.00576154]]
dW = [[ 0.10266786  0.09778551 -0.01968084]]
db = [[-0.05729622]]

relu:
dA_prev = [[ 0.44090989 -0.        ]
 [ 0.37883606 -0.        ]
 [-0.2298228   0.        ]]
dW = [[ 0.44513824  0.37371418 -0.10478989]]
db = [[-0.20837892]]


In [100]:
def L_model_backward(AL, Y, caches):
    grads = {}
    L = len(caches) 
    m = AL.shape[1]
    Y = Y.reshape(AL.shape) 
    
    dAL = - (np.divide(Y, AL) - np.divide(1 - Y, 1 - AL))
    
    current_cache = caches[L-1] 
    grads["dA" + str(L-1)], grads["dW" + str(L)], grads["db" + str(L)] = linear_activation_backward( dAL, current_cache, activation = "sigmoid" )

    # Loop de l=L-2 to l=0
    for l in reversed(range(L-1)):
        current_cache = caches[l]
        dA_prev_temp, dW_temp, db_temp = linear_activation_backward(grads["dA" + str(l+1)],current_cache, activation = "sigmoid")
        grads["dA" + str(l)] = dA_prev_temp
        grads["dW" + str(l + 1)] = dW_temp
        grads["db" + str(l + 1)] = db_temp
    return grads

In [101]:
#Probando la función
AL, Y_assess, caches = L_model_backward_test_case()
grads = L_model_backward(AL , Y_assess , caches)
print_grads(grads)

dW1 = [[ 0.0944987   0.01377483  0.03015356  0.02322328]
 [-0.09912664 -0.01366395 -0.03132431 -0.02417861]
 [ 0.01173121  0.0016263   0.00371069  0.00286357]]
db1 = [[-0.0357204 ]
 [ 0.03467639]
 [-0.00413664]]
dA1 = [[ 0.12913162 -0.44014127]
 [-0.14175655  0.48317296]
 [ 0.01663708 -0.05670698]]


In [111]:
def update_parameters(parameters, grads, learning_rate):
    L = len(parameters) // 2 # numero de capas de la red
    for l in range(1,(L+1)):
        parameters["W" + str(l)] =(parameters["W" + str(l)] - learning_rate * (grads["dW" + str(l)]) )
        parameters["b" + str(l)] =(parameters["b" + str(l)] - learning_rate * (grads["db" + str(l)]) )
    return parameters

In [112]:
parameters, grads = update_parameters_test_case()
parameters = update_parameters(parameters, grads, 0.1)
print ("W1 = "+ str(parameters["W1"]))
print ("b1 = "+ str(parameters["b1"]))
print ("W2 = "+ str(parameters["W2"]))
print ("b2 = "+ str(parameters["b2"]))

W1 = [[-0.59562069 -0.09991781 -2.14584584  1.82662008]
 [-1.76569676 -0.80627147  0.51115557 -1.18258802]
 [-1.0535704  -0.86128581  0.68284052  2.20374577]]
b1 = [[-0.04659241]
 [-1.28888275]
 [ 0.53405496]]
W2 = [[-0.55569196  0.0354055   1.32964895]]
b2 = [[-0.84610769]]


In [129]:
def nn_model(X, Y, layer_dims , num_iterations = 10000, learning_rate=1.2 , print_cost=False):
    np.random.seed(1)    
    parameters = initialize_parameters_deep(capas)
    costo=[]
    
    for i in range(0, num_iterations):
        AL , caches = L_model_forward(X, parameters) 
        #CALCULA FUNCION COSTO
        cost = compute_cost(AL , Y)
        #BACK PROGATION
        grads = L_model_backward(AL, Y, caches)
        #ACTUALIZACION DE PESOS 
        parameters = update_parameters(parameters, grads , learning_rate)
        if print_cost and i%100==0:
            print ("Costo despues iteracion de " + str(i) + ": " + str(cost))
        if  i % 1000 == 0:
            costo.append(cost)
    return parameters , costo

In [124]:
def predict(parameters, X):
    A2, cache = L_model_forward(X, parameters)
    predictions = np.rint(A2)
    return predictions

In [125]:
#OBTECION DE DATOS
def load_dataset():
    train_dataset = h5py.File('datasets/train_catvnoncat.h5', "r")
    train_set_x_orig = np.array(train_dataset["train_set_x"][:])
    train_set_y_orig = np.array(train_dataset["train_set_y"][:])
    test_dataset = h5py.File('datasets/test_catvnoncat.h5', "r")
    test_set_x_orig = np.array(test_dataset["test_set_x"][:])
    test_set_y_orig = np.array(test_dataset["test_set_y"][:])
    classes = np.array(test_dataset["list_classes"][:])
    train_set_y_orig = train_set_y_orig.reshape((1, train_set_y_orig.shape[0]))
    test_set_y_orig = test_set_y_orig.reshape((1, test_set_y_orig.shape[0]))#---
    return train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes

In [143]:
train_set_x_orig, train_set_y, test_set_x_orig, test_set_y, classes = load_dataset()
train_set_x_orig_flatten = train_set_x_orig.reshape (train_set_x_orig.shape [0], -1).T 
test_set_x_orig_flatten = test_set_x_orig.reshape (test_set_x_orig.shape [0], -1).T 

train_x_flatten = train_x_orig.reshape(train_x_orig.shape[0], -1).T  
test_x_flatten = test_x_orig.reshape(test_x_orig.shape[0], -1).T

train_set_x = train_set_x_orig_flatten/255
test_set_x = test_set_x_orig_flatten/255


In [127]:
layer_dims = [12288,1]

In [144]:
parameters , costo  = nn_model(train_set_x,  train_set_y , layer_dims  ,  num_iterations = 2500  , learning_rate = 0.0045 , print_cost=True )

W1 = (1, 12288)
b1 = (1, 1)
Costo despues iteracion de 0: 0.6909085741648417
Costo despues iteracion de 100: 0.5408600980416121
Costo despues iteracion de 200: 0.430827726697887
Costo despues iteracion de 300: 0.3831991254605257
Costo despues iteracion de 400: 0.347373094003985
Costo despues iteracion de 500: 0.3185769195240211
Costo despues iteracion de 600: 0.2946876523233277
Costo despues iteracion de 700: 0.2744182103744751
Costo despues iteracion de 800: 0.2569258158027438
Costo despues iteracion de 900: 0.24162754174329806
Costo despues iteracion de 1000: 0.2281032427605664
Costo despues iteracion de 1100: 0.21604017806616244
Costo despues iteracion de 1200: 0.2051994120132713
Costo despues iteracion de 1300: 0.19539441256558396
Costo despues iteracion de 1400: 0.18647687359377435
Costo despues iteracion de 1500: 0.1783270142185225
Costo despues iteracion de 1600: 0.1708467594986491
Costo despues iteracion de 1700: 0.1639548351597244
Costo despues iteracion de 1800: 0.15758316838

In [132]:
learning_rate = 0.00045
topologias  = [[12288,1] , [12288, 10 , 2 , 1] , [12288, 100 , 1] ,  [12288, 50 , 1]]

In [153]:
for i in topologias:
    parameters , costo  = nn_model(train_set_x,  train_set_y , i ,  num_iterations = 20000  , learning_rate = 0.004 , print_cost=False)
    
    Y_prediction_test = predict(parameters , test_set_x)
    Y_prediction_train = predict(parameters , train_set_x)
    
    print ("resultados en la topologia : "  + str(i))
    print("train accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_train - train_set_y)) * 100))
    print("test accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_test - test_set_y)) * 100))
    

resultados en la topologia : [12288, 1]
train accuracy: 100.0 %
test accuracy: 70.0 %
resultados en la topologia : [12288, 10, 2, 1]
train accuracy: 100.0 %
test accuracy: 70.0 %
resultados en la topologia : [12288, 100, 1]
train accuracy: 100.0 %
test accuracy: 70.0 %
resultados en la topologia : [12288, 50, 1]
train accuracy: 100.0 %
test accuracy: 70.0 %
